In [16]:
from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate, FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature = 0.1,
    streaming = True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ]
)

examples = [
    {
        "movie_name": "기생충",
        "director": "봉준호",
        "cast" : "송강호, 이선균, 조여정, 최우식, 박소담",
        "budget": "약 130억 원",
        "box_office" :"약 3,100억 원",
        "genre" : "드라마, 스릴러",
        "synopsis" : "기택의 가족은 전원이 백수로 생활고를 겪고 있다. 큰아들 기우는 친구의 소개로 부잣집 과외교사로 들어가며, 두 가족이 엮이게 된다.",
    },
    {
        "movie_name": "올드보이",
        "director": "박찬욱",
        "cast": "최민식, 유지태, 강혜정",
        "budget": "약 35억 원",
        "box_office": "약 175억 원",
        "genre": "스릴러, 미스터리",
        "synopsis": "15년간 이유도 모른 채 감금된 오대수는 갑자기 풀려나게 되고, 자신을 감금한 자를 찾아 복수를 시작한다."
    },

]


examples_prompt = PromptTemplate.from_template(
    template="""
    영화 이름: {movie_name}
    감독: {director}
    주요 출연진: {cast}
    예산: {budget}
    흥행 수익: {box_office}
    장르: {genre}
    시놉시스: {synopsis}
    """
)

prompt = FewShotPromptTemplate(
    example_prompt= examples_prompt,
    examples = examples,
    suffix = "영화 제목 : {input_movie_name}",
    input_variables=["input_movie_name"]
)

chain = prompt | chat

chain.invoke({
    "input_movie_name" : "부산행"
})


    감독: 연상호
    주요 출연진: 공유, 정유미, 마동석
    예산: 약 20억 원
    흥행 수익: 약 93억 원
    장르: 액션, 스릴러
    시놉시스: 전국적인 좀비 바이러스 감염으로 인해 기차를 타고 부산으로 향하는 사람들의 생존을 그린 영화.

AIMessageChunk(content='\n    감독: 연상호\n    주요 출연진: 공유, 정유미, 마동석\n    예산: 약 20억 원\n    흥행 수익: 약 93억 원\n    장르: 액션, 스릴러\n    시놉시스: 전국적인 좀비 바이러스 감염으로 인해 기차를 타고 부산으로 향하는 사람들의 생존을 그린 영화.')